<a href="https://colab.research.google.com/github/heriswn/LatihanDTS/blob/master/Google_Play_Store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as plt
import seaborn as sns

In [0]:
data=pd.read_csv("googleplaystore.csv")

In [84]:
data.head(2)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up


# Eksplorasi setiap feature
(data memiliki 12 tipe objek dan 1 tipe numerik)

1. Size
2. Installs
3. Reviews
4. Rating
5. Type
6. Price
7. Category
8. Content Rating
9. Genres
10. Last Updated
11. Current Version
12. Android Version

## Size

In [85]:
data.Size.value_counts()

Varies with device    1695
11M                    198
12M                    196
14M                    194
13M                    191
15M                    184
17M                    160
19M                    154
16M                    149
26M                    149
25M                    143
20M                    139
21M                    138
24M                    136
10M                    136
18M                    133
23M                    117
22M                    114
29M                    103
27M                     97
28M                     95
30M                     84
33M                     79
3.3M                    77
37M                     76
35M                     72
31M                     70
2.9M                    69
2.3M                    68
2.5M                    68
                      ... 
241k                     1
259k                     1
597k                     1
885k                     1
540k                     1
782k                     1
2

Dapat dilihat ada satuan kilo (k) dan Mega (M). Replace k dan M dengan nilai sesuai nilainya.

In [86]:
data.Size=data.Size.str.replace('k','e+3')
data.Size=data.Size.str.replace('M','e+6')
data.Size

0                     19e+6
1                     14e+6
2                    8.7e+6
3                     25e+6
4                    2.8e+6
5                    5.6e+6
6                     19e+6
7                     29e+6
8                     33e+6
9                    3.1e+6
10                    28e+6
11                    12e+6
12                    20e+6
13                    21e+6
14                    37e+6
15                   2.7e+6
16                   5.5e+6
17                    17e+6
18                    39e+6
19                    31e+6
20                    14e+6
21                    12e+6
22                   4.2e+6
23                   7.0e+6
24                    23e+6
25                   6.0e+6
26                    25e+6
27                   6.1e+6
28                   4.6e+6
29                   4.2e+6
                ...        
10811                3.9e+6
10812                 13e+6
10813                2.7e+6
10814                 31e+6
10815               

Kita punya dua tipe nilai dari data Size.
1. nilai exponensial (belum di convert ke string)
2. Strings (tidak bisa diconvert ke numerik)

untuk mengconvert category 1 ke numerik, jadikan category 1 dan 2 sebagai boolean array temp

In [87]:
def is_convertable(v):
    try:
        float(v)
        return True
    except ValueError:
        return False
    
temp=data.Size.apply(lambda x: is_convertable(x))
temp.head()

0    True
1    True
2    True
3    True
4    True
Name: Size, dtype: bool

Sekarang cek nilai non numerik

In [88]:
data.Size[~temp].value_counts()

Varies with device    1695
1,000+                   1
Name: Size, dtype: int64

* Replace 'Varies with device' oleh nan dan
* convert 1000+ to 1000 jadi numerik

In [0]:
data.Size=data.Size.replace('Varies with device',np.nan)
data.Size=data.Size.replace('1,000+',1000)

Convert data Size setelah dibersihkan

In [0]:
data.Size=pd.to_numeric(data.Size)

## Installs

Cek nilai unik dalam data install

In [91]:
data.Installs.shape

(10841,)

In [92]:
data.Installs.value_counts()

1,000,000+        1579
10,000,000+       1252
100,000+          1169
10,000+           1054
1,000+             907
5,000,000+         752
100+               719
500,000+           539
50,000+            479
5,000+             477
100,000,000+       409
10+                386
500+               330
50,000,000+        289
50+                205
5+                  82
500,000,000+        72
1+                  67
1,000,000,000+      58
0+                  14
0                    1
Free                 1
Name: Installs, dtype: int64

Dapat dilihat ada 22 nilai unik, yaitu
* 1 -> 0
* 1 -> free (sting)
* dan numerik cuma dengan tanda '+' dan ',' yang harus dihapus untuk mengconvert kedalam numerik

In [93]:
data.Installs=data.Installs.apply(lambda x: x.strip('+'))
data.Installs=data.Installs.apply(lambda x: x.replace(',',''))
data.Installs=data.Installs.replace('Free',np.nan)
data.Installs.value_counts()

1000000       1579
10000000      1252
100000        1169
10000         1054
1000           907
5000000        752
100            719
500000         539
50000          479
5000           477
100000000      409
10             386
500            330
50000000       289
50             205
5               82
500000000       72
1               67
1000000000      58
0               15
Name: Installs, dtype: int64

Cek data

In [94]:
data.Installs.str.isnumeric().sum()

10840

Artinya ada 1 data yang bukan numerik, yaitu nan (sudah diconvert dari Free ke nan)

In [95]:
data.Installs=pd.to_numeric(data.Installs)
data.Installs.head()

0       10000.0
1      500000.0
2     5000000.0
3    50000000.0
4      100000.0
Name: Installs, dtype: float64

## Reviews

Cek semua nilai dalam data reviews

In [96]:
data.Reviews.head()

0       159
1       967
2     87510
3    215644
4       967
Name: Reviews, dtype: object

In [97]:
data.Reviews.str.isnumeric().sum()

10840

Artinya ada 1 yang bukan numerik. Kita cari!

In [98]:
data[~data.Reviews.str.isnumeric()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,1000.0,NaN,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


Kita bisa saja mengubahnya menjadi numerik tetapi datanya aneh (M : jutaan).maka kita bisa drop (baris) data  

In [0]:
data=data.drop(data.index[10472])

cek baris

In [100]:
data[10471:].head(2)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10471,Xposed Wi-Fi-Pwd,PERSONALIZATION,3.5,1042,404000.0,100000.0,Free,0,Everyone,Personalization,"August 5, 2014",3.0.0,4.0.3 and up
10473,osmino Wi-Fi: free WiFi,TOOLS,4.2,134203,4100000.0,10000000.0,Free,0,Everyone,Tools,"August 7, 2018",6.06.14,4.4 and up


In [0]:
data.Reviews=data.Reviews.replace(data.Reviews[~data.Reviews.str.isnumeric()],np.nan)

In [0]:
data.Reviews=pd.to_numeric(data.Reviews)

## Rating

Nilai dari rating diantara 1-5. kita perlu pastikan dengan cara berikut

In [104]:
print("Range: ", data.Rating.min(),"-",data.Rating.max())

Range:  1.0 - 5.0


Cek jenis data untuk melihat apakah perlu dikonversi ke numerik

In [105]:
data.Rating.dtype

dtype('float64')

Data sudah numerik. sekarang cek data yang memiliki nilai null

In [106]:
print(data.Rating.isna().sum(),"null values out of", len(data.Rating))

1474 null values out of 10840


## Type

Cek tipe nilai dan masalah lain

In [108]:
data.Type.value_counts()

Free    10039
Paid      800
Name: Type, dtype: int64

Ada dua tipe, yaitu freee dan paid